# Metrics

## Import dependencies

In [1]:
import json
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import shutil
import math
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc
import subprocess
import csv
import seaborn as sns

## Preprocessing dataset

## Load dataset

In [2]:
DATA_SET_PATH = '/media/rodri/Files/Datasets/Roboflow_v11/Dataset'
DATA_SET_PATH

'/media/rodri/Files/Datasets/Roboflow_v11/Dataset'

## Generate annotations json

Headers:
- path
- class
- xmin
- ymin
- xmax
- ymax

Meanings:
- 0: gun
- 1: knife

In [5]:
# Read one file
def read_yolo_file(path):
    with open(path, 'r') as f:
        aux = f.readlines()
        aux = [x.strip().replace(' ',',') for x in aux]
        aux = [x.split(',') for x in aux]
        aux = [[int(x[0]), float(x[1]), float(x[2]), float(x[3]), float(x[4])] for x in aux]
        
        def convert_yolo_format_to_voc(labels, img_width, img_height):
            """ Convert Yolo format to VOC format

            Args:
                x (list): [class, x, y, w, h]
                img_width (int): Image width
                img_height (int): Image height

            Returns:
                list: [class, xmin, ymin, xmax, ymax]
            """
            labels = labels.copy()
            
            x = labels[1] * img_width
            y = labels[2] * img_height
            w = labels[3] * img_width
            h = labels[4] * img_height
            
            xmin = math.floor(x - (w / 2)+1)
            ymin = math.floor(y - (h / 2)+1)
            xmax = math.floor(x + (w / 2)+1)
            ymax = math.floor(y + (h / 2)+1)
            
            return [labels[0], xmin, ymin, xmax, ymax]
        
        def get_image_shape(path):
            img = cv2.imread(path)
            # Getting image path
            try:
                img_path = os.path.dirname(os.path.dirname(path)) + '/images/' + os.path.basename(path).replace('.txt', '.jpg')
                # print(img_path)
                img = cv2.imread(img_path)
                return img.shape[1], img.shape[0]
            finally:
                del img_path
                del img
        
        
        aux = [convert_yolo_format_to_voc(x, *get_image_shape(path)) for x in aux]
        
        # Check if annotations are empty
        if len(aux) == 0:
            aux.append([-1, 0, 0, 0, 0])
            
        [x.insert(0, path) for x in aux]
        
        
        def convert_to_json(x):
            """ Convert to json format
            Consider new meanings:
            - 0: background
            - 1: gun
            - 2: knife

            Args:
                x (list): List of lists with the following format: [path, class, x1, y1, x2, y2]

            Returns:
                _type_: Description of return value
            """        
            
            return {
                'path': x[0],
                'class': x[1]+1,
                'bbox':{
                    'xmin': x[2],
                    'ymin': x[3],
                    'xmax': x[4],
                    'ymax': x[5]
                }
            }
        
        aux = [convert_to_json(x) for x in aux]
        
        # return file name and annotations
        return aux

# Test cases

## No annotations
path = os.path.join(DATA_SET_PATH, 'test/labels/batch_39_jpg.rf.5d98d73855063aadf8a2ddee468b970a.txt')
print("No annotations", end=': ')
print(read_yolo_file(path))

## One annotation
path = os.path.join(DATA_SET_PATH, 'test/labels/00000668_jpg.rf.2031d4cedcf31cd67c48cff1d053eddf.txt')
print("One annotation", end=': ')
print(read_yolo_file(path))

## Multiple annotations
path = os.path.join(DATA_SET_PATH, 'test/labels/batch_40_jpg.rf.d083c576c8cf66569711b7eab35f2df5.txt')
print("Multiple annotations", end=': ')
print(read_yolo_file(path))


No annotations: [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/batch_39_jpg.rf.5d98d73855063aadf8a2ddee468b970a.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}]
One annotation: [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/00000668_jpg.rf.2031d4cedcf31cd67c48cff1d053eddf.txt', 'class': 1, 'bbox': {'xmin': 18, 'ymin': 53, 'xmax': 611, 'ymax': 395}}]
Multiple annotations: [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/batch_40_jpg.rf.d083c576c8cf66569711b7eab35f2df5.txt', 'class': 1, 'bbox': {'xmin': 2, 'ymin': 5, 'xmax': 312, 'ymax': 314}}, {'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/batch_40_jpg.rf.d083c576c8cf66569711b7eab35f2df5.txt', 'class': 1, 'bbox': {'xmin': 397, 'ymin': 68, 'xmax': 610, 'ymax': 297}}, {'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/batch_40_jpg.rf.d083c576c8cf66569711b7eab35f2df5.txt', 'class': 1, 'bbox': {'xmin':

In [6]:
test_labels = [read_yolo_file(os.path.join(DATA_SET_PATH, 'test/labels', x)) for x in tqdm(os.listdir(os.path.join(DATA_SET_PATH, 'test/labels')), desc='Reading test labels')]

Reading test labels: 100%|██████████| 1294/1294 [00:30<00:00, 42.33it/s] 


In [7]:
print(f'Test labels: {len(test_labels)}')
print(f'Last 5 test labels: {json.dumps(test_labels[-5:], indent=4)}')

Test labels: 1294
Last 5 test labels: [
    [
        {
            "path": "/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000295310.txt",
            "class": 0,
            "bbox": {
                "xmin": 0,
                "ymin": 0,
                "xmax": 0,
                "ymax": 0
            }
        }
    ],
    [
        {
            "path": "/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000303738.txt",
            "class": 0,
            "bbox": {
                "xmin": 0,
                "ymin": 0,
                "xmax": 0,
                "ymax": 0
            }
        }
    ],
    [
        {
            "path": "/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000307618.txt",
            "class": 0,
            "bbox": {
                "xmin": 0,
                "ymin": 0,
                "xmax": 0,
                "ymax": 0
            }
        }
    ],
    [
        {
            "path": "/media/rodri/Files/D

## Save annotations as json files

In [8]:
json.dump(test_labels, open(os.path.join(DATA_SET_PATH, 'test_labels.json'), 'w'))

In [9]:
del test_labels

## Fix json

In [7]:
test_labels = json.load(open(os.path.join(DATA_SET_PATH, 'test_labels.json'), 'r'))

### Convert annotation path from labels to image path

In [8]:
def convert_label_path_to_image_path(json: list) -> list:
    """ Convert label path to image path

    Args:
        json (list): Json to convert

    Returns:
        list: Converted json
    """
    for item in tqdm(json, desc='Converting label path to image path'):
        for subitem in item:
            subitem['path'] = (subitem['path'].replace('labels', 'images'))[:-3] + 'jpg'
    return json

# Before
print(test_labels[5:])

# Test case
print(convert_label_path_to_image_path(test_labels[5:]))

[[{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000010023.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000010701.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000022056.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000023687.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000024414.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/labels/000000031082.txt', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Robofl

Converting label path to image path: 100%|██████████| 1289/1289 [00:00<00:00, 624302.29it/s]

[[{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/images/000000010023.jpg', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/images/000000010701.jpg', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/images/000000022056.jpg', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/images/000000023687.jpg', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/images/000000024414.jpg', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Roboflow_v11/Dataset/test/images/000000031082.jpg', 'class': 0, 'bbox': {'xmin': 0, 'ymin': 0, 'xmax': 0, 'ymax': 0}}], [{'path': '/media/rodri/Files/Datasets/Robofl

In [9]:
test_labels = convert_label_path_to_image_path(test_labels)

Converting label path to image path: 100%|██████████| 1294/1294 [00:00<00:00, 586749.12it/s]


### Save results as json files on metrics folder

In [11]:
json.dump(test_labels, open(os.path.join(DATA_SET_PATH, 'test_labels.json'), 'w'))